Goals
1. Download all zippped files and unzip csv files to 2 seperate folders. One for Ford and one for Lyft - Do in automated fashion if possible
2. Combine each of the 10+ csv files into 2 big files, one for Ford and one for Lyft
3. Push to AWS S3

Bonus: Each month write a code that auto pulls the new zipped file from the cloud and appends it to our existing Lyft AWS dataset

In [57]:
#I will utilize some code from my past web scraping project and tweak it for this project.
#I will use selenium since that is what I have used in the past and is what I am familiar with

#This currently looks up certain fields for you and enters them into the website. 
#Next would be to automate this to run through all 4 designations I want. Do that at end. Also make cleaner later.
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
import math
import regex as re 
from selenium.webdriver.support.ui import Select

from zipfile import ZipFile
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from decimal import Decimal
import numpy as np
import pandas as pd
import math
import os
import io

from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics import r2_score
import boto3 


In [35]:
#This will auto download all the code currently on baywheels website

#set download path
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : r"C:\Users\gadde\Desktop\Data_Analytics_projects\baywheels_analysis\zipped_data"}
chromeOptions.add_experimental_option("prefs",prefs)

#start chromedriver
driver = webdriver.Chrome(chrome_options=chromeOptions)
driver.get('https://s3.amazonaws.com/baywheels-data/index.html')
#//*[@id="tbody-content"]/tr[1]/td[1]/a
time.sleep(1)

#Note we have it hard set to 53. Ideally we could use a lenght of something to automatically determine this but I don't
#know how to do that.
for i in range(53):
    download = driver.find_element(by=By.XPATH, value='//*[@id="tbody-content"]/tr[' + str(i+1) + ']/td[1]/a')
    download.click()

#Close out driver
driver.quit()

C:\Users\gadde\AppData\Local\Temp/ipykernel_10500/713211868.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chromeOptions)


In [44]:
#now let us go ahead and unzip all the files in the zipped_data folder
for root, dirs, files in os.walk('zipped_data'):
    for name in files:
        if 'ford' in name:
            with ZipFile(os.path.join(root, name), 'r') as zip_ref:
                zip_ref.extractall('baywheels_data\Ford')
        elif 'baywheels' in name:
            with ZipFile(os.path.join(root, name), 'r') as zip_ref:
                zip_ref.extractall('baywheels_data\Lyft')
        else:
            print('Error')

In [52]:
#Now lets combine the multiple csv's in each of these folders into one large df and output that to a csv
path = 'baywheels_data\Ford'
ford_file_names = []
for files in os.listdir(path):
    if 'csv' in files:
        ford_data.append(os.path.join(path, files))
        #print(os.path.join(path, files))
        
#Load the csv into a dataframe
df_ford = pd.concat(map(pd.read_csv, ford_file_names))


baywheels_data\Ford\2017-fordgobike-tripdata.csv
baywheels_data\Ford\201801-fordgobike-tripdata.csv
baywheels_data\Ford\201802-fordgobike-tripdata.csv
baywheels_data\Ford\201803-fordgobike-tripdata.csv
baywheels_data\Ford\201804-fordgobike-tripdata.csv
baywheels_data\Ford\201805-fordgobike-tripdata.csv
baywheels_data\Ford\201806-fordgobike-tripdata.csv
baywheels_data\Ford\201807-fordgobike-tripdata.csv
baywheels_data\Ford\201808-fordgobike-tripdata.csv
baywheels_data\Ford\201809-fordgobike-tripdata.csv
baywheels_data\Ford\201810-fordgobike-tripdata.csv
baywheels_data\Ford\201811-fordgobike-tripdata.csv
baywheels_data\Ford\201812-fordgobike-tripdata.csv
baywheels_data\Ford\201901-fordgobike-tripdata.csv
baywheels_data\Ford\201902-fordgobike-tripdata.csv
baywheels_data\Ford\201903-fordgobike-tripdata.csv
baywheels_data\Ford\201904-fordgobike-tripdata.csv


In [55]:
#repeat above step for Lyft data
path = 'baywheels_data\Lyft'
lyft_file_names = []
for files in os.listdir(path):
    if 'csv' in files:
        lyft_file_names.append(os.path.join(path, files))
        #print(os.path.join(path, files))
        
#Load the csv into a dataframe
df_lyft = pd.concat(map(pd.read_csv, lyft_file_names))

baywheels_data\Lyft\201905-baywheels-tripdata.csv
baywheels_data\Lyft\201906-baywheels-tripdata.csv
baywheels_data\Lyft\201907-baywheels-tripdata.csv
baywheels_data\Lyft\201908-baywheels-tripdata.csv
baywheels_data\Lyft\201909-baywheels-tripdata.csv
baywheels_data\Lyft\201910-baywheels-tripdata.csv
baywheels_data\Lyft\201911-baywheels-tripdata.csv
baywheels_data\Lyft\201912-baywheels-tripdata.csv
baywheels_data\Lyft\202001-baywheels-tripdata.csv
baywheels_data\Lyft\202002-baywheels-tripdata.csv
baywheels_data\Lyft\202003-baywheels-tripdata.csv
baywheels_data\Lyft\202004-baywheels-tripdata.csv
baywheels_data\Lyft\202005-baywheels-tripdata.csv
baywheels_data\Lyft\202006-baywheels-tripdata.csv
baywheels_data\Lyft\202007-baywheels-tripdata.csv
baywheels_data\Lyft\202008-baywheels-tripdata.csv
baywheels_data\Lyft\202009-baywheels-tripdata.csv
baywheels_data\Lyft\202010-baywheels-tripdata.csv
baywheels_data\Lyft\202011-baywheels-tripdata.csv
baywheels_data\Lyft\202012-baywheels-tripdata.csv


C:\Users\gadde\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:294: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(
C:\Users\gadde\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:294: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(
C:\Users\gadde\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:294: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(


In [60]:
#Write the ford and lyft data to a csv
df_ford.to_csv(r'baywheels_data\ford_data.csv')
df_lyft.to_csv(r'baywheels_data\lyft_data.csv')

In [63]:
#Lastly, let us upload these 2 large dataframes in an S3 bucket. We will upload this using the csv format.
#I know that are probably more efficient file storage formats but since this data isn't huge csv should suffice

s3 = boto3.client("s3")

s3.upload_file(
    Filename=r'baywheels_data\ford_data.csv',
    Bucket="baywheels.data",
    Key="ford_data.csv",
)

In [ ]:
#Same for Lyft data

s3 = boto3.client("s3")

s3.upload_file(
    Filename=r'baywheels_data\lyft_data.csv',
    Bucket="baywheels.data",
    Key="lyft_data.csv",
)